In [2]:
#Use only on colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install pandas==1.5.1 transformers==4.26.0 torch==1.13.1 datasets==2.9.0 seqeval==1.2.2 evaluate==0.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 15.8 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3

In [4]:
import pandas as pd

# specify the filepath of the Excel file
filepath = r'/content/drive/MyDrive/Full Data For Text Extraction.xlsx'

# read the Excel file into a DataFrame
df = pd.read_excel(filepath, nrows=200000)

# display the first five rows of the DataFrame
df.head()

,review_text_index,review,keywords,label,parts,Unnamed: 5
0,36,Good ingredients so it doesn't irritate my ...,smell,fragrance (negative/unneeded),Not a fan of the smell at all,NaN
1,36,Good ingredients so it doesn't irritate my ...,"hydrat, moistur",hydrating/moisturising,skin feels super hydrated but not oily... Foll...,NaN
2,36,Good ingredients so it doesn't irritate my ...,mask,mask,Only face sheet mask I've actually seen make a...,NaN
3,36,Good ingredients so it doesn't irritate my ...,irritat,no irritation,so it doesn't irritate my skin,NaN
4,36,Good ingredients so it doesn't irritate my ...,oil,oil free finish,skin feels super hydrated but not oily,NaN


In [5]:
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')

def preprocess(df, column = 'review'):

    if column == 'review':
      #drop duplicate rows
      df = df.drop_duplicates()

      #drop rows where na values present in parts column
      if 'parts' in df.columns:
        df = df.dropna(subset=['parts'])

    # lowercase the text
    df[column] = df[column].str.lower()
    
    #add septoken
    if 'parts' in df.columns and column == 'parts':
        df[column] = df[column].apply(lambda x: str(x).replace("...", " septoken ") if '...' in str(x) else str(x) + " septoken ")

    # remove special characters and numbers
    df[column] = df[column].apply(lambda x: re.sub(r'[^\w\s]+', '', str(x)))

    # remove underscores
    df[column] = df[column].apply(lambda x: x.replace("_", "") if '_' in x else x)

    # tokenize the reviews
    df[column] = df[column].apply(lambda x: word_tokenize(x))
    
    
    return df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
df = preprocess(df)
df = preprocess(df, column ='parts')

In [7]:
df = df.explode('review').reset_index()

In [9]:
def categorize(review, parts):
    categories = []
    i = 0
    while i < len(review):
        category = ["O"]
        if parts[i][-1] == 'septoken':
            if review[i] == parts[i][0] and len(parts[i]) > 1 and " ".join(review[i:i+len(parts[i])-1]) == " ".join(parts[i][:-1]):
                category1 = ["B-Part"]
                category2 = ["I-Part"] * (len(parts[i]) - 2)
                category = category1 + category2
                i += len(parts[i]) - 2
        else:
            sepind = parts[i].index('septoken')
            if review[i] == parts[i][0] and len(parts[i]) > 1 and " ".join(review[i:i+sepind]) == " ".join(parts[i][:sepind]):
                category1 = ["B-Part"]
                category2 = ["I-Part"] * (sepind - 1)
                category = category1 + category2
                i += sepind - 1
            elif review[i] == parts[i][sepind + 1] and len(parts[i]) > 1 and " ".join(review[i:i + len(parts[i]) - sepind - 1]) == " ".join(parts[i][sepind + 1:]):
                category1 = ["B-Part"]
                category2 = ["I-Part"] * (len(parts[i]) - sepind - 2)
                category = category1 + category2
                i += len(parts[i]) - sepind - 2
        categories.extend(category)
        i += 1
    return categories

df["category"] = categorize(df["review"], df["parts"])

In [10]:
df1 = df.groupby(['review_text_index', 'label']).agg({'review':lambda x: " ".join(list(x)), 'category':lambda x: list(x)}).reset_index()

In [11]:
df1.head()

,review_text_index,label,review,category
0,36,fragrance (negative/unneeded),good ingredients so it doesnt irritate my skin...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,36,hydrating/moisturising,good ingredients so it doesnt irritate my skin...,"[O, O, O, O, O, O, O, O, O, O, O, B-Part, I-Pa..."
2,36,mask,good ingredients so it doesnt irritate my skin...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,36,no irritation,good ingredients so it doesnt irritate my skin...,"[O, O, B-Part, I-Part, I-Part, I-Part, I-Part,..."
4,36,oil free finish,good ingredients so it doesnt irritate my skin...,"[O, O, O, O, O, O, O, O, O, O, O, B-Part, I-Pa..."


In [12]:
# Function to remove special characters and spaces from a string
def clean_string(string):
    cleaned_string = re.sub(r'[^\w]', '', string) # Removes special characters and spaces including underscores
    return cleaned_string

# Apply the function to the 'label' column
df1['label'] = df1['label'].apply(clean_string)
df1['reviews'] =  df1['review'] + " " + df1['label']

In [13]:
#check tokenizer of bert
from transformers import AutoTokenizer
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [14]:
#tokenizer.is_fast

In [15]:
inputs = tokenizer(df1["reviews"][0])
list(inputs.tokens())

['[CLS]',
 'good',
 'ingredients',
 'so',
 'it',
 'doesn',
 '##t',
 'i',
 '##rri',
 '##tate',
 'my',
 'skin',
 'healthy',
 'glow',
 'afterwards',
 'skin',
 'feels',
 'super',
 'h',
 '##yd',
 '##rated',
 'but',
 'not',
 'oil',
 '##y',
 'only',
 'face',
 'sheet',
 'mask',
 'i',
 '##ve',
 'actually',
 'seen',
 'make',
 'a',
 'difference',
 'not',
 'a',
 'fan',
 'of',
 'the',
 'smell',
 'at',
 'all',
 'follow',
 'up',
 'with',
 'c',
 '##lini',
 '##que',
 'gel',
 'moist',
 '##uri',
 '##ser',
 'with',
 'couple',
 'drops',
 'of',
 'face',
 'hero',
 'oil',
 'f',
 '##rag',
 '##rance',
 '##ne',
 '##gat',
 '##ive',
 '##un',
 '##nee',
 '##ded',
 '[SEP]']

In [16]:
inputs.word_ids()

[None,
 0,
 1,
 2,
 3,
 4,
 4,
 5,
 5,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 14,
 14,
 15,
 16,
 17,
 17,
 18,
 19,
 20,
 21,
 22,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 39,
 39,
 40,
 41,
 41,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 49,
 None]

In [17]:
label_names = ['O','B-Part', 'I-Part']
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [18]:
def label2ids(label, label2id):
    return label2id[label]

In [19]:
from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
encoder = LabelEncoder()
unique_labels = np.array(['O', 'B-Part', 'I-Part'])
# Fit the encoder to the 'category' column
encoder.fit(unique_labels)
# Transform the 'category' column into numbers
df1['category'] = df1['category'].apply(lambda x: x + ['O'])
df1['category_encoded'] = df1['category'].apply(lambda x: [label2ids(x, label2id) for x in x])

In [20]:
df1['category_encoded'][0]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [21]:
#Align labels with tokens
#For Each separated token from same word, same label is given
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [22]:
labels = df1["category_encoded"][1]
inputs = tokenizer(df1["reviews"][1])
word_ids = inputs.word_ids()
print(labels) #print original labels
print(align_labels_with_tokens(labels, word_ids)) #print aligned labels with tokens

[None, 0, 1, 2, 3, 4, 4, 5, 5, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 14, 14, 15, 16, 17, 17, 18, 19, 20, 21, 22, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 39, 39, 40, 41, 41, 41, 42, 43, 44, 45, 46, 47, 48, 49, 49, 49, 49, 49, 49, 49, None]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0]
['[CLS]', 'good', 'ingredients', 'so', 'it', 'doesn', '##t', 'i', '##rri', '##tate', 'my', 'skin', 'healthy', 'glow', 'afterwards', 'skin', 'feels', 'super', 'h', '##yd', '##rated', 'but', 'not', 'oil', '##y', 'only', 'face', 'sheet', 'mask', 'i', '##ve', 'actually', 'seen', 'make', 'a', 'difference', 'not', 'a', 'fan', 'of', 'the', 'smell', 'at', 'all', 'follow', 'up', 'with', 'c', '##lini', '##que', 'gel', 'moist', '##uri', '##ser', 'with', 'couple', 'drops', 'of', 'face', 'hero', 'oil', 'h', '##yd', '##rating', '##mo', '##ist', '##uri', '##sing', '[SEP]']
[-100, 0, 0, 0, 

In [23]:
#Tokenize align labels of entire dataframe
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["reviews"], truncation=True)
    all_labels = examples["category_encoded"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [24]:
# Train-Test split
from sklearn.model_selection import train_test_split
y = df1['category_encoded']
x_train,x_test = train_test_split(df1,test_size=0.2,random_state=42)

In [25]:
#create traning and validation data in DatasetDict format
from datasets import Dataset, DatasetDict
tds = Dataset.from_pandas(x_train)
vds = Dataset.from_pandas(x_test)

ds = DatasetDict()

ds['train'] = tds
ds['validation'] = vds
ds

DatasetDict({
    train: Dataset({
        features: ['review_text_index', 'label', 'review', 'category', 'reviews', 'category_encoded', '__index_level_0__'],
        num_rows: 157840
    })
    validation: Dataset({
        features: ['review_text_index', 'label', 'review', 'category', 'reviews', 'category_encoded', '__index_level_0__'],
        num_rows: 39460
    })
})

In [26]:
#Create Dataset in batched data format
tokenized_datasets = ds.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=ds["train"].column_names,
)
tokenized_datasets

  0%|          | 0/158 [00:00<?, ?ba/s]

  0%|          | 0/40 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 157840
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 39460
    })
})

In [27]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [28]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch['labels']

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    1,    2,    2,    2,
            2,    2,    2,    2,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0, -100],
        [-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    1,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
         -100, -100, -100, -100, -100, -100, 

In [30]:
import numpy as np
import evaluate
metric = evaluate.load("seqeval")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [31]:
#Build Model
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [32]:
model.config.num_labels

3

In [33]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [34]:
from transformers import TrainingArguments

args = TrainingArguments(
    "/content/drive/MyDrive/bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=15,
    weight_decay=0.01,
    push_to_hub=False,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model.to(device),
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

#use below code only when you want to resume training from specific checkpoint
#trainer.train('/content/drive/MyDrive/bert-finetuned-ner/checkpoint-98650')

Loading model from /content/drive/MyDrive/bert-finetuned-ner/checkpoint-98650.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 157840
  Num Epochs = 15
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 295950
  Number of trainable parameters = 107721987
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 5
  Continuing training from global step 98650
  Will skip the first 5 epochs then the first 0 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch co

0it [00:00, ?it/s]

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
6,0.039900,0.122399,0.769212,0.794464,0.781634,0.972906


***** Running Evaluation *****
  Num examples = 39460
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/bert-finetuned-ner/checkpoint-118380
Configuration saved in /content/drive/MyDrive/bert-finetuned-ner/checkpoint-118380/config.json
Model weights saved in /content/drive/MyDrive/bert-finetuned-ner/checkpoint-118380/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/bert-finetuned-ner/checkpoint-118380/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/bert-finetuned-ner/checkpoint-118380/special_tokens_map.json
